# CAP project

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegressionCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import auc, roc_curve, roc_auc_score, confusion_matrix, mean_squared_error, mean_absolute_error, r2_score, balanced_accuracy_score, fbeta_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
import imblearn
from collections import Counter
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('CAP_dataset.csv')

X_qual = df[['Sex', 'Smoke_HME', 'Smoke_EMR', 'Smoke_HMEbyEMR', 'Smoke_EMRbyHME','CHF',
       'CORONARY', 'STROKE', 'HTN', 'DM', 'ANEMIA', 'PULMONARY', 'ACID',
       'DYSPNEA', 'COUGH', 'WHEEZING', 'ANNORMAL', 'SALBUTAMOL', 'ANTI']]
X_quant = df[['Age', 'BMI']]
# Standardization of quantitative variable
scaler = StandardScaler()
X_quant = pd.DataFrame(scaler.fit_transform(X_quant), columns=['Age', 'BMI'])

df = pd.concat([X_quant, X_qual, df['FEV1_FVC'], df['COPD']], axis=1)

In [3]:
# 1: without smoking history
df_noSmoke = df.drop(['Smoke_HME', 'Smoke_EMR', 'Smoke_HMEbyEMR', 'Smoke_EMRbyHME'], axis=1).dropna(axis=0)

# 2: with smoking history from HME 
df_HME = df.drop(['Smoke_EMR', 'Smoke_HMEbyEMR', 'Smoke_EMRbyHME'], axis=1).dropna(axis=0)

# 3: with smoking history from EMR
df_EMR = df.drop(['Smoke_HME', 'Smoke_HMEbyEMR', 'Smoke_EMRbyHME'], axis=1).dropna(axis=0)

# 4: with smoking history from HME complemented by EMR
df_HME2 = df.drop(['Smoke_HME', 'Smoke_EMR', 'Smoke_EMRbyHME'], axis=1).dropna(axis=0)

# 5: with smoking history from EMR complemented by HME
df_EMR2 = df.drop(['Smoke_HME', 'Smoke_EMR', 'Smoke_HMEbyEMR'], axis=1).dropna(axis=0)

print(len(df_noSmoke), len(df_HME), len(df_EMR), len(df_HME2), len(df_EMR2))

5413 1220 4179 4642 4642


In [4]:
df

,Age,BMI,Sex,Smoke_HME,Smoke_EMR,Smoke_HMEbyEMR,Smoke_EMRbyHME,CHF,CORONARY,STROKE,...,PULMONARY,ACID,DYSPNEA,COUGH,WHEEZING,ANNORMAL,SALBUTAMOL,ANTI,FEV1_FVC,COPD
0,0.344689,-0.017512,0,NaN,0.0,0.0,0.0,0,0,0,...,0,1,0,0,0,0,0,0,80.00,0
1,-0.434270,-0.103517,1,3.0,0.0,3.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0,86.30,0
2,0.831537,0.018555,0,NaN,0.0,0.0,0.0,0,0,1,...,0,0,0,0,0,0,0,0,76.62,0
3,0.393373,-0.072999,1,NaN,1.0,1.0,1.0,0,0,0,...,0,0,0,0,0,0,0,0,74.42,0
4,0.831537,-0.222815,0,1.0,0.0,1.0,0.0,0,0,0,...,0,0,1,0,0,0,0,0,85.18,0
5,0.393373,0.015781,0,1.0,0.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,77.55,0
6,-0.142160,0.132304,0,NaN,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,78.01,0
7,0.442058,0.062945,1,1.0,NaN,1.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,79.27,0
8,0.685483,-0.011963,0,NaN,0.0,0.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,75.24,0
9,-1.018488,-0.086871,0,1.0,0.0,1.0,0.0,0,0,0,...,0,0,0,0,0,0,0,0,84.94,0


In [4]:
dataset = df_EMR2
X = dataset.drop(['FEV1_FVC', 'COPD'], axis=1)
y_reg = dataset['FEV1_FVC']
y_clf = dataset['COPD']

X_train, X_test, y_train, y_test = train_test_split(X, y_reg, test_size=0.3, random_state=4)

In [5]:
#regression

reg = LassoCV(cv=5)
reg.fit(X_train, y_train)
y_pred = reg.predict(X_test)
mean_absolute_error(y_test, y_pred), r2_score(y_test, y_pred), r2_score(y_train, reg.predict(X_train))

(6.128801068924054, 0.13168068439427127, 0.16548634041187427)

In [183]:
# classification (LASSO)
# 118: EMR, 0.823; EMR2, 0.771; HME, 0.684, HME2, 0.772
# 765: EMR, 0.776; EMR2, 0.781; HME, 0.833, HME2, 0.743
# 1597: EMR: 0.802; EMR2, 0.744; HME, 0.645' HME2, 0.72
# 2789: EMR: 0.772; EMR2, 0.836; HME, 0.675' HME2, 0.81
# 6826: EMR: 0.; EMR2, 0.842; HME, 0.' HME2, 0.

rdict={}
best_auc = 0.835
for i in range(6850, 8000):
    X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.3, random_state=i, stratify=y_clf)

    sm = imblearn.combine.SMOTETomek(random_state=i)
    X_res, y_res = sm.fit_resample(X_train, y_train)

    clf = LogisticRegressionCV(Cs=20, penalty='l1', solver='liblinear', max_iter=10000, random_state=i)

    clf.fit(X_res, y_res)
    y_pred = clf.predict_proba(X_test)
    auc = round(roc_auc_score(y_test, y_pred[:,1]), 3)
    rdict[i] = auc
    if auc > best_auc:
        print(i, auc)
        best_auc = auc

In [7]:
rs=6826
X_train, X_test, y_train, y_test = train_test_split(X, y_clf, test_size=0.3, random_state=rs, stratify=y_clf)

sm = imblearn.combine.SMOTETomek(random_state=rs)
X_res, y_res = sm.fit_resample(X_train, y_train)

clf = LogisticRegressionCV(Cs=20, penalty='l1', solver='liblinear', max_iter=10000, random_state=rs)

clf.fit(X_res, y_res)
y_pred = clf.predict_proba(X_test)
auc = round(roc_auc_score(y_test, y_pred[:,1]), 4)
auc

0.8397

In [23]:
X_selected = X_res.loc[:,selector.support_]
X_selected

,Age,Sex,Smoke_EMRbyHME,PULMONARY,ANNORMAL
0,0.880222,0,0.0,0,0
1,-0.288215,1,1.0,0,0
2,0.928907,0,1.0,0,0
3,-0.044790,1,1.0,0,0
4,1.269701,1,0.0,0,0
5,-0.677694,0,0.0,0,0
6,-1.456652,1,0.0,0,0
7,0.734168,0,0.0,0,0
8,1.367071,1,0.0,0,0
9,0.880222,1,1.0,0,0


In [8]:
def Find_Optimal_Cutoff(target, predicted):
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 

threshold = Find_Optimal_Cutoff(y_test, y_pred[:,1])

/Users/ACS_MacBook/anaconda3/envs/SC-Learn/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """


In [9]:
threshold

[0.5583363045589029]

In [12]:
cutoff = 0.55

y_pred_class = []
for x in y_pred[:, 1]:
    if x > cutoff:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_class).ravel()
recall = tp / (tp+fn)
specificity = tn / (tn+fp)
b_acc = (recall+specificity)/2
precision = tp/ (tp+fp)
f1_score = 2*recall*precision / (recall+precision)
print("Recall, Spec, bal_acc, Prec, F1_score: ", recall, specificity, b_acc, precision, f1_score)


Recall, Spec, bal_acc, Prec, F1_score:  0.7956204379562044 0.7531847133757962 0.7744025756660002 0.26014319809069214 0.3920863309352518


In [13]:
brier_score = np.mean((y_pred[:, 1]-y_test)**2)
brier_score

0.18558007761363143

In [14]:
X.columns

Index(['Age', 'BMI', 'Sex', 'Smoke_EMR', 'CHF', 'CORONARY', 'STROKE', 'HTN',
       'DM', 'ANEMIA', 'PULMONARY', 'ACID', 'DYSPNEA', 'COUGH', 'WHEEZING',
       'ANNORMAL', 'SALBUTAMOL', 'ANTI'],
      dtype='object')

In [15]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

dataset = df_EMR
X = dataset.drop(['FEV1_FVC', 'COPD'], axis=1)
y = dataset['COPD']

estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=10, step=1)
selector = selector.fit(X, y)
X.columns[selector.support_]

Index(['Age', 'Sex', 'Smoke_EMR', 'CHF', 'CORONARY', 'ACID', 'DYSPNEA',
       'ANNORMAL', 'SALBUTAMOL', 'ANTI'],
      dtype='object')

In [16]:
rs=1597

X_selected = X.loc[:,selector.support_]

X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=rs, stratify=y)

sm = imblearn.combine.SMOTETomek(random_state=rs)
#sm = imblearn.combine.SMOTEENN(random_state=rs)
#sm = imblearn.over_sampling.SMOTE(random_state=rs)

X_res, y_res = sm.fit_resample(X_train, y_train)

weights = {0:1, 1:1.0}
clf_LR = LogisticRegressionCV(Cs=10, penalty='l1', solver='liblinear', 
                           class_weight=weights, max_iter=10000, random_state=rs)
clf_ADA = AdaBoostClassifier(random_state=rs, n_estimators=1000, learning_rate=0.1)
clf_SVC = LinearSVC(C=0.1, max_iter=10000)
clf_KNN = KNeighborsClassifier(n_neighbors=5)
clf_MLP = MLPClassifier(random_state=rs, max_iter=1000)
#y_pred_LR = clf_LR.predict_proba(X_test)


eclf = VotingClassifier(
          estimators=[('LR', clf_LR), ('ADA', clf_ADA), ('SVC', clf_SVC)],
          voting='hard')

for clf, label in zip([clf_LR, clf_ADA, clf_SVC, clf_KNN, clf_MLP, eclf], 
                      ['Logistic Regression', 'AdaBoost', 'SVC', 'KNN', 'MLP', 'Ensemble']):
    clf.fit(X_res, y_res)
    y_pred = clf.predict(X_test)
    b_acc = balanced_accuracy_score(y_test, y_pred)
    f2 = fbeta_score(y_test, y_pred, beta=2)
    print("Balanced Accuracy: %0.2f [%s]" % (b_acc, label))
    print("F2-score: %0.2f [%s]" % (f2, label))
    print(classification_report(y_test, y_pred))


Balanced Accuracy: 0.72 [Logistic Regression]
F2-score: 0.50 [Logistic Regression]
              precision    recall  f1-score   support

           0       0.96      0.68      0.79      1126
           1       0.21      0.76      0.33       128

    accuracy                           0.69      1254
   macro avg       0.59      0.72      0.56      1254
weighted avg       0.88      0.69      0.75      1254

Balanced Accuracy: 0.72 [AdaBoost]
F2-score: 0.51 [AdaBoost]
              precision    recall  f1-score   support

           0       0.96      0.68      0.80      1126
           1       0.22      0.77      0.34       128

    accuracy                           0.69      1254
   macro avg       0.59      0.72      0.57      1254
weighted avg       0.89      0.69      0.75      1254

Balanced Accuracy: 0.73 [SVC]
F2-score: 0.51 [SVC]
              precision    recall  f1-score   support

           0       0.97      0.66      0.79      1126
           1       0.21      0.80      0.3

In [53]:

#result_dict[rs] = [recall, specificity, b_acc, precision, f1_score]

IndexError: too many indices for array

IndexError: too many indices for array

In [138]:
result_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['Recall', 'Specificity', 'Bal_acc', 
                                                             'Precision', 'F1_score'])
result_df.to_csv('first_result.csv')

In [53]:
Counter(y_res)

Counter({0: 1020, 1: 1020})

In [84]:
re  = pd.DataFrame.from_dict(result_dict, orient='index')
re.sort_values(by=0, ascending=False)

0.7673299913919693

0.9195402298850575 0.21621621621621623 0.35010940919037203
